In [27]:
pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [28]:
import logging
import json

from FhirClient import FhirClient

logging.basicConfig()
logging.getLogger().setLevel(logging.WARNING)

In [29]:
def setup_fhir_client() -> FhirClient:
    fhir_client = FhirClient(base_url='https://concept01.ehealthexchange.org:52780/fhirproxy/r4/',
                        verify_ssl=False,
                        extra_headers={
                            'X-DESTINATION': 'PublicHAPI',
                            'X-POU': 'TREAT',
                            'Host': 'concept01.ehealthexchange.org:52780'
                        })
    
    fhir_client.auth_type = 'Basic'
    fhir_client.token = 'c3ZjX2VIeEZISVJTYW5kYm94OndpbGxmdWxTdHJvbmdTdGFuZHVyZDc'

    return fhir_client

In [30]:
client = setup_fhir_client()
search_results = client.search('Patient', {
    'family': 'Sanity',
    'gender': 'male',
    'address-state': 'MH',
})

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'concept01.ehealthexchange.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [36]:
patient = search_results['entry'][0]
patient

{'fullUrl': 'https://concept01.ehealthexchange.org:52780/fhirproxy/r4/Patient/592578',
 'resource': {'resourceType': 'Patient',
  'id': '592578',
  'meta': {'versionId': '3',
   'lastUpdated': '2023-09-01T21:40:21.291+00:00',
   'source': '#Dr3UQWhWSQXRlglj'},
  'text': {'status': 'generated',
   'div': '<div xmlns="http://www.w3.org/1999/xhtml"><div class="hapiHeaderText">TestforPatientR4 <b>SANITY </b></div><table class="hapiPropertyTable"><tbody><tr><td>Date of birth</td><td><span>03 November 1919</span></td></tr></tbody></table></div>'},
  'name': [{'family': 'Sanity', 'given': ['TestforPatientR4']}],
  'gender': 'male',
  'birthDate': '1919-11-03',
  'address': [{'city': 'Pune', 'district': 'Pune', 'state': 'MH'}],
  'identifier': [{'use': 'usual',
    'type': {'text': 'Target responder resource fullUrl'},
    'system': 'https://ehealthexchange.org/hub/internal/targetResponderFullUrl',
    'value': 'https://hapi.fhir.org/baseR4/Patient/592578'}]},
 'search': {'mode': 'match'}}

In [32]:
client.search('MedicationAdministration', query_params={
            'patient': patient['resource']['id']
})

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'concept01.ehealthexchange.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'resourceType': 'Bundle',
 'id': 'd6d645fa-7bfc-4ff3-bb5b-fa5a1c7332c7',
 'meta': {'lastUpdated': '2024-01-10T18:31:05.298+00:00'},
 'type': 'searchset',
 'total': 0,
 'link': [{'relation': 'self',
   'url': 'https://hapi.fhir.org/baseR4/MedicationAdministration?patient=592578'}]}

In [33]:
client.search('Observation', query_params={
            'patient': patient['resource']['id']
})

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'concept01.ehealthexchange.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'resourceType': 'Bundle',
 'id': 'bf834e31-45c8-4fcb-8892-bcf71eb00b56',
 'meta': {'lastUpdated': '2024-01-10T18:31:06.298+00:00'},
 'type': 'searchset',
 'total': 4,
 'link': [{'relation': 'self',
   'url': 'https://hapi.fhir.org/baseR4/Observation?patient=592578'}],
 'entry': [{'fullUrl': 'https://concept01.ehealthexchange.org:52780/fhirproxy/r4/Observation/10306938',
   'resource': {'resourceType': 'Observation',
    'id': '10306938',
    'meta': {'versionId': '1',
     'lastUpdated': '2023-04-19T09:59:29.950+00:00',
     'source': '#oYLJWp5ROmXi9xOP'},
    'code': {'coding': [{'system': 'http://foo', 'code': '12345'}]},
    'subject': {'reference': 'Patient/592578'},
    'effectiveDateTime': '2023-04-19T09:34:50+01:00',
    'valueQuantity': {'value': 123,
     'unit': 'kg',
     'system': 'http://bar',
     'code': 'kg'},
    'identifier': [{'use': 'usual',
      'type': {'text': 'Target responder resource fullUrl'},
      'system': 'https://ehealthexchange.org/hub/internal/target

In [34]:
client.search('Encounter', query_params={
            'patient': patient['resource']['id']
})

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'concept01.ehealthexchange.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'resourceType': 'Bundle',
 'id': '08609c78-809d-46f7-927c-11977ccbce5a',
 'meta': {'lastUpdated': '2024-01-10T18:31:07.298+00:00'},
 'type': 'searchset',
 'total': 0,
 'link': [{'relation': 'self',
   'url': 'https://hapi.fhir.org/baseR4/Encounter?patient=592578'}]}

In [35]:
client.close()